^fill^README.md^here^

^fill^theory.md^here^

## Demonstration

### 1. Setup

#### 1.1. Library imports

Import libraries needed by the calculation. The external libraries used are:

- [numpy](http://www.numpy.org/)

- [DataModelDict](https://github.com/usnistgov/DataModelDict)

- [atomman](https://github.com/usnistgov/atomman)

- [iprPy](https://github.com/usnistgov/iprPy)

In [ ]:
# Standard library imports
from pathlib import Path
import os
import sys
import uuid
import shutil
import datetime
import random
from copy import deepcopy

# http://www.numpy.org/
import numpy as np

# https://github.com/usnistgov/DataModelDict 
from DataModelDict import DataModelDict as DM

# https://github.com/usnistgov/atomman 
import atomman as am
import atomman.lammps as lmp
import atomman.unitconvert as uc

# https://github.com/usnistgov/iprPy
import iprPy

print('Notebook last executed on', datetime.date.today(), 'using iprPy version', iprPy.__version__)

#### 1.2. Default calculation setup

In [ ]:
# Specify calculation style
calc_style = 'relax_dynamic'

# If workingdir is already set, then do nothing (already in correct folder)
try:
    workingdir = workingdir

# Change to workingdir if not already there
except:
    workingdir = Path('calculationfiles', calc_style)
    if not workingdir.is_dir():
        workingdir.mkdir(parents=True)
    os.chdir(workingdir)

### 2. Assign values for the calculation's run parameters

#### 2.1. Specify system-specific paths

- __lammps_command__ is the LAMMPS command to use (required).

- __mpi_command__ MPI command for running LAMMPS in parallel. A value of None will run simulations serially.

In [ ]:
lammps_command = 'lmp_serial'
mpi_command = None

#### 2.2. Load interatomic potential

- __potential_name__ gives the name of the potential_LAMMPS reference record in the iprPy library to use for the calculation.  

- __potential_file__ gives the path to the potential_LAMMPS reference record to use.  Here, this parameter is automatically generated using potential_name and librarydir.

- __potential_dir__ gives the path for the folder containing the artifacts associated with the potential (i.e. eam.alloy file).  Here, this parameter is automatically generated using potential_name and librarydir.

- __potential__ is an atomman.lammps.Potential object (required).  Here, this parameter is automatically generated from potential_file and potential_dir.

In [ ]:
potential_name = '1999--Mishin-Y--Ni--LAMMPS--ipr1'

# Define potential_file and potential_dir using librarydir and potential_name
potential_file = Path(iprPy.libdir, 'potential_LAMMPS', f'{potential_name}.json')
potential_dir = Path(iprPy.libdir, 'potential_LAMMPS', potential_name)

# Initialize Potential object using potential_file and potential_dir.
potential = lmp.Potential(potential_file, potential_dir)
print('Successfully loaded potential', potential)

#### 2.3. Load initial unit cell system

- __prototype_name__ gives the name of the crystal_prototype reference record in the iprPy library to load. 

- __symbols__ is a list of the potential's elemental model symbols to associate with the unique atom types of the loaded system. 

- __box_parameters__ is a list of the a, b, c lattice constants to assign to the loaded file.

- __load_file__ gives the path to the atomic configuration file to load for the ucell system.  Here, this is generated automatically using prototype_name and librarydir.

- __load_style__ specifies the format of load_file.  Here, this is automatically set for crystal_prototype records.

- __load_options__ specifies any other keyword options for properly loading the load_file.  Here, this is automatically set for crystal_prototype records.

- __ucell__ is an atomman.System representing a fundamental unit cell of the system (required).  Here, this is generated using the load parameters and symbols.

In [ ]:
prototype_name = 'A1--Cu--fcc'
symbols = ['Ni']
box_parameters = uc.set_in_units([3.5, 3.5, 3.5], 'angstrom')

# Define load_file using librarydir and prototype_name
load_file = Path(iprPy.libdir, 'crystal_prototype', f'{prototype_name}.json')

# Define load_style and load_options for crystal_prototype records
load_style = 'system_model'
load_options = {}

# Create ucell by loading prototype record
ucell = am.load(load_style, load_file, symbols=symbols, **load_options)

# Rescale ucell using box_parameters
ucell.box_set(a=box_parameters[0], b=box_parameters[1], c=box_parameters[2], scale=True)

print(ucell)

#### 2.4. Modify system

- __sizemults__ list of three integers specifying how many times the ucell vectors of $a$, $b$ and $c$ are replicated in creating system.

- __system__ is an atomman.System to perform the scan on (required). 

In [ ]:
sizemults = [10, 10, 10]

# Generate system by supersizing ucell
system = ucell.supersize(*sizemults)
print('# of atoms in system =', system.natoms)

#### 2.5. Specify calculation-specific run parameters

- __pressure_xx__ gives the xx component of the pressure to equilibriate the system to (npt, nph, and nph+l styles).

- __pressure_yy__ gives the yy component of the pressure to equilibriate the system to (npt, nph, and nph+l styles).

- __pressure_zz__ gives the zz component of the pressure to equilibriate the system to (npt, nph, and nph+l styles).

- __pressure_xy__ gives the xy component of the pressure to equilibriate the system to (npt, nph, and nph+l styles).

- __pressure_xz__ gives the xz component of the pressure to equilibriate the system to (npt, nph, and nph+l styles).

- __pressure_yz__ gives the yz component of the pressure to equilibriate the system to (npt, nph, and nph+l styles).

- __temperature__ gives the temperature to equilibriate the system to (nvt, npt, nve+l, and nph+l styles).

- __integrator__ specifies the integrator style to use. Default value is 'nph+l' for temperature = 0, and 'npt' otherwise.

- __runsteps__ is the total number of integration timesteps to perform. Default value is 220000.

- __thermosteps__ specifies to output thermo values every this many timesteps. Default value is 100.
    
- __dumpsteps__ specifies to output dump files every this many timesteps. Default value is runsteps (only first and last steps are outputted as dump files).
    
- __equilsteps__ is the number of timesteps to equilibriate the system for. Only thermo values associated with timesteps greater than equilsteps will be included in the mean and standard deviation calculations. Default value is 20000. 

- __randomseed__ specifies a random number seed used to generate the initial atomic velocities and the Langevin thermostat fluctuations. Default value generates a new random integer every time.

In [ ]:
pressure_xx = uc.set_in_units(0.0, 'GPa')
pressure_yy = uc.set_in_units(0.0, 'GPa')
pressure_zz = uc.set_in_units(0.0, 'GPa')
pressure_xy = uc.set_in_units(0.0, 'GPa')
pressure_xz = uc.set_in_units(0.0, 'GPa')
pressure_yz = uc.set_in_units(0.0, 'GPa')
temperature = 300.0
integrator = 'npt'
runsteps = 220000
thermosteps = 100
dumpsteps = runsteps
equilsteps = 20000
randomseed = None

### 3. Define calculation function(s) and generate template LAMMPS script(s)

#### 3.1. full_relax.template

In [ ]:
with open('full_relax.template', 'w') as f:
    f.write("""^fill^full_relax.template^here^""")

#### 3.2. integrator_info()

In [ ]:
^fill^calc_relax_dynamic.py(integrator_info)^here^

#### 3.3. full_relax()

In [ ]:
^fill^calc_relax_dynamic.py(relax_dynamic)^here^

### 4. Run calculation function(s)

In [ ]:
results_dict = relax_dynamic(lammps_command, system, potential,
                             mpi_command = mpi_command,
                             p_xx = pressure_xx,
                             p_yy = pressure_yy,
                             p_zz = pressure_zz,
                             p_xy = pressure_xy,
                             p_xz = pressure_xz,
                             p_yz = pressure_yz,
                             temperature = temperature,
                             runsteps = runsteps,
                             integrator = integrator,
                             thermosteps = thermosteps,
                             dumpsteps = dumpsteps,
                             equilsteps = equilsteps,
                             randomseed = randomseed)

In [ ]:
results_dict.keys()

### 5. Report results

#### 5.1. Define units for outputting values

- __length_unit__ is the unit of length to display values in.
- __energy_unit__ is the unit of energy to display values in.
- __pressure_unit__ is the unit of pressure to display values in.

In [ ]:
length_unit = 'angstrom'
energy_unit = 'eV'
pressure_unit = 'GPa'

#### 5.2. Display number of samples

In [ ]:
print('# of samples = ', results_dict['nsamples'])

#### 5.3. Print mean and standard deviations of measured values

In [ ]:
print('Measured thermo data with standard deviation:')
print('Ecoh = %9f +- %9f %s' % (uc.get_in_units(results_dict['E_coh'], energy_unit),
                                    uc.get_in_units(results_dict['E_coh_std'], energy_unit),
                                    energy_unit))
print('lx   = %9f +- %9f %s' % (uc.get_in_units(results_dict['lx'], length_unit),
                                    uc.get_in_units(results_dict['lx_std'], length_unit),
                                    length_unit))
print('ly   = %9f +- %9f %s' % (uc.get_in_units(results_dict['ly'], length_unit),
                                    uc.get_in_units(results_dict['ly_std'], length_unit),
                                    length_unit))     
print('lz   = %9f +- %9f %s' % (uc.get_in_units(results_dict['lz'], length_unit),
                                    uc.get_in_units(results_dict['lz_std'], length_unit),
                                    length_unit)) 
print('xy   = %9f +- %9f %s' % (uc.get_in_units(results_dict['xy'], length_unit),
                                    uc.get_in_units(results_dict['xy_std'], length_unit),
                                    length_unit))
print('xz   = %9f +- %9f %s' % (uc.get_in_units(results_dict['xz'], length_unit),
                                    uc.get_in_units(results_dict['xz_std'], length_unit),
                                    length_unit))     
print('yz   = %9f +- %9f %s' % (uc.get_in_units(results_dict['yz'], length_unit),
                                    uc.get_in_units(results_dict['yz_std'], length_unit),
                                    length_unit)) 
print('T    = %9f +- %9f %s' % (results_dict['temp'],results_dict['temp_std'], 'K')) 
print('Pxx  = %9f +- %9f %s' % (uc.get_in_units(results_dict['measured_pxx'], pressure_unit),
                                    uc.get_in_units(results_dict['measured_pxx_std'], pressure_unit),
                                    pressure_unit)) 
print('Pyy  = %9f +- %9f %s' % (uc.get_in_units(results_dict['measured_pyy'], pressure_unit),
                                    uc.get_in_units(results_dict['measured_pyy_std'], pressure_unit),
                                    pressure_unit)) 
print('Pzz  = %9f +- %9f %s' % (uc.get_in_units(results_dict['measured_pzz'], pressure_unit),
                                    uc.get_in_units(results_dict['measured_pzz_std'], pressure_unit),
                                    pressure_unit)) 
print('Pxy  = %9f +- %9f %s' % (uc.get_in_units(results_dict['measured_pxy'], pressure_unit),
                                    uc.get_in_units(results_dict['measured_pxy_std'], pressure_unit),
                                    pressure_unit)) 
print('Pxz  = %9f +- %9f %s' % (uc.get_in_units(results_dict['measured_pxz'], pressure_unit),
                                    uc.get_in_units(results_dict['measured_pxz_std'], pressure_unit),
                                    pressure_unit)) 
print('Pyz  = %9f +- %9f %s' % (uc.get_in_units(results_dict['measured_pyz'], pressure_unit),
                                    uc.get_in_units(results_dict['measured_pyz_std'], pressure_unit),
                                    pressure_unit)) 

#### 5.4. Compute lattice constant with standard error

**NOTE**: This step makes two assumptions

1. The crystal structure is cubic and remains cubic after relaxation.  Check values above to verify this.

2. The thermosteps parameter is large enough that the measurements are not correlated. If thermosteps &ge; 100 this is likely a sound assumption.

In [ ]:
a = results_dict['lx'] / sizemults[0]
b = results_dict['ly'] / sizemults[1]
c = results_dict['lz'] / sizemults[2]

a_std = results_dict['lx_std'] / sizemults[1]
b_std = results_dict['ly_std'] / sizemults[2]
c_std = results_dict['lz_std'] / sizemults[2]

a_mean = (a + b + c) / 3
a_combined_std = ((a_std**2 + b_std**2 + c_std**2 
                   + (a - a_mean)**2 + (b - a_mean)**2 + (c - a_mean)**2) / 3)**0.5
a_standard_error = a_combined_std * (3 * results_dict['nsamples'])**-0.5

print('Cubic lattice constant with standard error:')
print('a = %9f +- %9f %s' % (uc.get_in_units(a_mean, length_unit),
                             uc.get_in_units(a_standard_error, length_unit),
                             length_unit))

#### 5.5. Load final configuration and show box

In [ ]:
finalsystem = am.load('atom_dump', results_dict['dumpfile_final'],
                      symbols=results_dict['symbols_final'])
print(finalsystem.box)